# 0. Introduction
1. We have two `class` to extract the information neighbors from center atoms, they are all derived class of `StructureNeighborsBase`
    1. `pflow.io.publicLayer.neigh.StructureNeighborsV1`: Use `NearestNeighbors` in `sklearn`
    2. `pflow.io.publicLayer.neigh.StructureNeighborsV2`: Implemet custom `NearestNeighbors`
    3. Now `StructureNeighborsV2` is much faster than `StructureNeighborsV1`

In [5]:
from timeit import default_timer as timer
from pflow.io.publicLayer.structure import DStructure
from pflow.io.publicLayer.neigh import StructureNeighborsV1
from pflow.io.publicLayer.neigh import StructureNeighborsV2

# 1. Demo for `StructureNeighborsV1`
1. Running time: 1.66 s

In [9]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/pflow/demo/structure/atom.config"
scaling_matrix = [5, 5, 1]  # Expand the primitive cell to process pbc
reformat_mark = True    # Resort the atoms according to atomic number after expanding to supercell
n_neighbors = 200   # The total number of all kinds of atoms. You'd better set it larger than `max_num_nbr`(usually 100) in `DeepPot-SE`
algorithm = "ball_tree" # The algorithm for `sklearn.NearestNerghbors`
coords_are_cartesian = True # Use cartesian coordinates to find neighbors

_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
struct_neighbors = StructureNeighborsV1(
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    n_neighbors=n_neighbors,
                    algorithm=algorithm)
_end = timer()

print("\t1.1. The number of atoms in primitive cell:\t", len(struct_neighbors.structure.species))
print("\t1.2. The shape of key_nbr_atomic_numbers:\t", struct_neighbors.key_nbr_atomic_numbers.shape)
print("\t1.3. The shape of key_nbr_distances:\t", struct_neighbors.key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", struct_neighbors.key_nbr_coords.shape)
print("Running time: {0} s".format(_end-_start))

	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_atomic_numbers:	 (12, 200)
	1.3. The shape of key_nbr_distances:	 (12, 200)
	1.4. The shape of key_nbr_coords:	 (12, 200, 3)
Running time: 1.4815923422574997 s


# 2. Demo for `StructureNeighborsV2`
1. Running time: 0.037 s

In [11]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/pflow/demo/structure/atom.config"
scaling_matrix = [5, 5, 1]  # Expand the primitive cell to process pbd
reformat_mark = True    # Resort the atoms according to atomic number after expanding to supercell
n_neighbors = 200   # The total number of all kinds of atoms. You'd better set it larger than `max_num_nbr`(usually 100) in `DeepPot-SE`
coords_are_cartesian = True # Use cartesian coordinates to find neighbors

_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
struct_neighbors = StructureNeighborsV2(
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    n_neighbors=n_neighbors)
_end = timer()

print("\t1.1. The number of atoms in primitive cell:\t", len(struct_neighbors.structure.species))
print("\t1.2. The shape of key_nbr_atomic_numbers:\t", struct_neighbors.key_nbr_atomic_numbers.shape)
print("\t1.3. The shape of key_nbr_distances:\t", struct_neighbors.key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", struct_neighbors.key_nbr_coords.shape)
print("Running time: {0} s".format(_end-_start))

	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_atomic_numbers:	 (12, 200)
	1.3. The shape of key_nbr_distances:	 (12, 200)
	1.4. The shape of key_nbr_coords:	 (12, 200, 3)
Running time: 0.027402549982070923 s
